In [1]:
import pandas as pd 
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import models,layers

In [2]:
path="C:\\Users\\andyr\\Documents\\file\\side\\projects\\stock\\sp500\\csv"
os.chdir(path)
data=pd.read_csv(os.path.join(path,"AAPL.csv"))

In [3]:
data

,Date,Low,Open,Volume,High,Close,Adjusted Close
0,12-12-1980,0.128348,0.128348,469033600,0.128906,0.128348,0.100326
1,15-12-1980,0.121652,0.122210,175884800,0.122210,0.121652,0.095092
2,16-12-1980,0.112723,0.113281,105728000,0.113281,0.112723,0.088112
3,17-12-1980,0.115513,0.115513,86441600,0.116071,0.115513,0.090293
4,18-12-1980,0.118862,0.118862,73449600,0.119420,0.118862,0.092911
...,...,...,...,...,...,...,...
10396,08-03-2022,155.800003,158.820007,131148300,162.880005,157.440002,157.440002
10397,09-03-2022,159.410004,161.479996,91454900,163.410004,162.949997,162.949997
10398,10-03-2022,155.979996,160.199997,105342000,160.389999,158.520004,158.520004
10399,11-03-2022,154.500000,158.929993,96836300,159.279999,154.729996,154.729996


In [4]:
X=data["Open"]

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10401 entries, 0 to 10400
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            10401 non-null  object 
 1   Low             10401 non-null  float64
 2   Open            10401 non-null  float64
 3   Volume          10401 non-null  int64  
 4   High            10401 non-null  float64
 5   Close           10401 non-null  float64
 6   Adjusted Close  10401 non-null  float64
dtypes: float64(5), int64(1), object(1)
memory usage: 568.9+ KB


In [6]:
#did same data preprocessing steps in previous file
data["Date"] = pd.to_datetime(data["Date"])

In [7]:
data=data.set_index('Date')

In [8]:
from sklearn.preprocessing import MinMaxScaler
#scale data for neural network

In [9]:
mms=MinMaxScaler()

In [10]:
data_scaled=mms.fit_transform(data)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
data=data[["Open","Close"]]

In [13]:
train_size=round(len(data)*0.8)

In [14]:
train_size

8321

In [15]:
train_data=data[:train_size]

In [16]:
test_data=data[train_size:]

In [17]:
train_data.shape,test_data.shape

((8321, 2), (2080, 2))

In [18]:
def create_target(train_data):
    seq=[]
    target=[]
    start=0
    for stop in range(50,len(train_data)):
        seq.append(train_data.iloc[start:stop])
        target.append(train_data.iloc[stop])
        start+=1
    return (np.array(seq),np.array(target))





In [19]:
train_seq,train_labels=create_target(train_data)

In [20]:
test_seq,test_labels=create_target(test_data)

In [21]:
train_seq.shape,train_labels.shape,test_seq.shape,test_labels.shape

((8271, 50, 2), (8271, 2), (2030, 50, 2), (2030, 2))

In [22]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional
#apply LSTM to the data

In [23]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape = (train_seq.shape[1], train_seq.shape[2])))
model.add(Dropout(0.1)) 
model.add(LSTM(units=50))
model.add(Dense(2))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 50)            10600     
                                                                 
 dropout (Dropout)           (None, 50, 50)            0         
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 2)                 102       
                                                                 
Total params: 30,902
Trainable params: 30,902
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(train_seq,train_labels,epochs=10,validation_data=(test_seq,test_labels))

Epoch 1/10
259/259 [==============================] - 8s 22ms/step - loss: 7.1164 - mean_absolute_error: 0.8015 - val_loss: 3933.4878 - val_mean_absolute_error: 45.7098
Epoch 2/10
259/259 [==============================] - 5s 20ms/step - loss: 0.9015 - mean_absolute_error: 0.2625 - val_loss: 3632.0728 - val_mean_absolute_error: 42.1701
Epoch 3/10
259/259 [==============================] - 5s 20ms/step - loss: 0.2740 - mean_absolute_error: 0.1605 - val_loss: 3514.3240 - val_mean_absolute_error: 40.3394
Epoch 4/10
259/259 [==============================] - 5s 20ms/step - loss: 0.1171 - mean_absolute_error: 0.1186 - val_loss: 3417.6946 - val_mean_absolute_error: 39.1868
Epoch 5/10
259/259 [==============================] - 5s 20ms/step - loss: 0.0781 - mean_absolute_error: 0.1084 - val_loss: 3326.3669 - val_mean_absolute_error: 38.3133
Epoch 6/10
259/259 [==============================] - 5s 20ms/step - loss: 0.0592 - mean_absolute_error: 0.0944 - val_loss: 3261.2256 - val_mean_absolute_e

In [25]:
model.predict(test_seq)

array([[19.119644, 19.255756],
       [18.882301, 18.997444],
       [18.775848, 18.888155],
       ...,
       [23.41629 , 23.489824],
       [23.418346, 23.491545],
       [23.42075 , 23.493597]], dtype=float32)

In [26]:
test_labels

array([[ 19.0282135 ,  18.7589283 ],
       [ 18.68392944,  18.84107018],
       [ 18.90642929,  18.64500046],
       ...,
       [160.19999695, 158.52000427],
       [158.92999268, 154.72999573],
       [151.44999695, 151.68499756]])